In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import os, sys
%matplotlib inline

os.chdir('..\..\..\..')

from input_configuration import base_year

from IPython.display import display, HTML

if (sys.version_info < (3, 0)):
    HTML('''<script>
    code_show=true; 
    function code_toggle() {
     if (code_show){
     $('div.input').hide();
     } else {
     $('div.input').show();
     }
     code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
working_dir = r'outputs/agg/dash'
# pd.options.display.float_format = '{:0,.0f}'.format
format_percent = "{:,.2%}".format

In [3]:
def load_data(fname):
    """ open and join model and survey data """

    df_model = pd.read_csv(os.path.join(working_dir,fname+'.csv'))
    df_model['source'] = 'model'
    df_survey = pd.read_csv(os.path.join(working_dir,'survey',fname+'.csv'))
    df_survey['source'] = 'survey'
    df = df_model.append(df_survey)

    return df

In [4]:
parcel_geog = pd.read_sql_table('parcel_'+base_year+'_geography', 'sqlite:///inputs/db/soundcast_inputs.db',
        columns=['CountyName','ParcelID'])

In [5]:
def dist_by_mode(purpose=None, county_filter='', neg_county_filter=False):
    df = load_data('tour_distance')
    if county_filter!='' :
        if neg_county_filter:
            df = df.loc[df.person_county!=county_filter]
        else:
            df = df.loc[df.person_county==county_filter]            
    # Trim any unreasonable survey distances off by taking max distance as max modeled bin value
    max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
    df = df.loc[df['tautodist_bin'] <= max_model_dist]
    df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
    df_dist = pd.DataFrame()
    for source in ['model','survey']:
        if purpose is not None:
            _df = df[(df['source'] == source) & (df['pdpurp'] == purpose)]
        else:
            _df = df[(df['source'] == source)]
        df1 = _df.groupby('tmodetp').sum()[['wt_autodist']].reset_index()
        df2 = _df.groupby('tmodetp').sum()[['toexpfac']].reset_index ()
        _df = df1.merge(df2, on='tmodetp')
        _df['commute_distance'] = _df['wt_autodist']/_df['toexpfac']
        _df['source'] = source
        df_dist = df_dist.append(_df)
    df = df_dist.pivot_table(index='tmodetp', columns='source', aggfunc='first', values='commute_distance')
    df.rename(columns={'tmodetp': 'Tour Mode'}, inplace=True)
    for col in ['School Bus',' Park']:
        if (col in df.index) & (purpose != 'School'):
            df.drop(col, axis=0, inplace=True)
        if (col != 'School Bus') & (col in df.index):
            df.drop(col, axis=0, inplace=True)
    
    return df

### Total

In [6]:
df = load_data('tour_distance')
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df = df[(df['pdpurp'] == 'School')]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df = df.groupby('source').sum()
pd.DataFrame(df['wt_autodist']/df['toexpfac'], columns=['School Distance (miles)'])

,School Distance (miles)
source,
model,3.701011
survey,4.283469


## Pierce County

In [7]:
df = load_data('tour_distance')
df = df.loc[df['person_county']=='Pierce']
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df = df[(df['pdpurp'] == 'School')]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df = df.groupby('source').sum()
pd.DataFrame(df['wt_autodist']/df['toexpfac'], columns=['School Distance (miles)'])

,School Distance (miles)
source,
model,3.733780
survey,4.106343


## Other Counties

In [8]:
df = load_data('tour_distance')
df = df.loc[df['person_county']!='Pierce']
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df = df[(df['pdpurp'] == 'School')]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df = df.groupby('source').sum()
pd.DataFrame(df['wt_autodist']/df['toexpfac'], columns=['School Distance (miles)'])

,School Distance (miles)
source,
model,3.692216
survey,4.326237


### By Purpose

In [9]:
df = load_data('tour_distance')
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df_dist = pd.DataFrame()
for source in ['model','survey']:
    _df = df[(df['source'] == source)]
    df1 = _df.groupby('pdpurp').sum()[['wt_autodist']].reset_index()
    df2 = _df.groupby('pdpurp').sum()[['toexpfac']].reset_index ()
    _df = df1.merge(df2, on='pdpurp')
    _df['commute_distance'] = _df['wt_autodist']/_df['toexpfac']
    _df['source'] = source
    df_dist = df_dist.append(_df)
df = df_dist.pivot_table(index='pdpurp', columns='source', aggfunc='first', values='commute_distance')
df.rename(columns={'pdpurp': 'Tour Purpose'}, inplace=True)
df.drop('Change Mode Inserted Purpose', axis=0, inplace=True)
df

source,model,survey
pdpurp,,
Escort,7.043379,3.884350
Meal,5.368908,4.223924
Personal Business,6.770857,5.486210
School,3.701011,4.283469
Shop,4.762047,4.527161
Social,4.499123,4.889143
Work,11.814426,11.508843


## Pierce County

In [10]:
df = load_data('tour_distance')
df = df.loc[df['person_county']=='Pierce']
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df_dist = pd.DataFrame()
for source in ['model','survey']:
    _df = df[(df['source'] == source)]
    df1 = _df.groupby('pdpurp').sum()[['wt_autodist']].reset_index()
    df2 = _df.groupby('pdpurp').sum()[['toexpfac']].reset_index ()
    _df = df1.merge(df2, on='pdpurp')
    _df['commute_distance'] = _df['wt_autodist']/_df['toexpfac']
    _df['source'] = source
    df_dist = df_dist.append(_df)
df = df_dist.pivot_table(index='pdpurp', columns='source', aggfunc='first', values='commute_distance')
df.rename(columns={'pdpurp': 'Tour Purpose'}, inplace=True)
df.drop('Change Mode Inserted Purpose', axis=0, inplace=True, errors='ignore')
df

source,model,survey
pdpurp,,
Escort,7.216835,4.710262
Meal,6.120633,5.531909
Personal Business,7.087517,6.912622
School,3.733780,4.106343
Shop,5.348092,4.446887
Social,4.657890,3.897583
Work,14.726366,12.672461


## Other Counties

In [11]:
df = load_data('tour_distance')
df = df.loc[df['person_county']!='Pierce']
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df_dist = pd.DataFrame()
for source in ['model','survey']:
    _df = df[(df['source'] == source)]
    df1 = _df.groupby('pdpurp').sum()[['wt_autodist']].reset_index()
    df2 = _df.groupby('pdpurp').sum()[['toexpfac']].reset_index ()
    _df = df1.merge(df2, on='pdpurp')
    _df['commute_distance'] = _df['wt_autodist']/_df['toexpfac']
    _df['source'] = source
    df_dist = df_dist.append(_df)
df = df_dist.pivot_table(index='pdpurp', columns='source', aggfunc='first', values='commute_distance')
df.rename(columns={'pdpurp': 'Tour Purpose'}, inplace=True)
df.drop('Change Mode Inserted Purpose', axis=0, inplace=True, errors='ignore')
df

source,model,survey
pdpurp,,
Escort,6.995048,3.615977
Meal,5.218580,3.884169
Personal Business,6.682658,5.183605
School,3.692216,4.326237
Shop,4.605618,4.548941
Social,4.459262,5.099560
Work,11.204138,11.226877


# Tour Distance
### All Tours

In [12]:
dist_by_mode()

source,model,survey
tmodetp,,
Bike,5.351260,3.032501
HOV2,7.305732,6.698868
HOV3+,6.930744,6.216336
Park,16.026210,24.393105
SOV,8.845273,8.497786
TNC,2.513342,3.992971
Transit,9.550902,9.402977
Walk,0.930132,0.889081


## Pierce County

In [13]:
dist_by_mode(county_filter='Pierce')

source,model,survey
tmodetp,,
Bike,5.716311,4.779864
HOV2,7.666877,7.037761
HOV3+,7.133040,6.737807
Park,22.329932,33.465259
SOV,9.786382,8.717633
TNC,2.777231,10.342347
Transit,18.447401,15.274353
Walk,1.030344,0.922871


## Other Counties

In [14]:
dist_by_mode(county_filter='Pierce', neg_county_filter=True)

source,model,survey
tmodetp,,
Bike,5.268515,2.844943
HOV2,7.208704,6.613760
HOV3+,6.874720,6.085078
Park,14.604798,23.033155
SOV,8.613114,8.435609
TNC,2.467930,3.599855
Transit,8.775368,8.955929
Walk,0.910506,0.883049


### Average Work Tour Distance By Mode

In [15]:
dist_by_mode('Work')

source,model,survey
tmodetp,,
Bike,5.765433,4.307769
HOV2,11.849042,11.350996
HOV3+,12.283944,15.006290
Park,16.026210,23.257848
SOV,12.416810,11.779248
TNC,6.217664,4.858663
Transit,12.926918,10.774342
Walk,0.756235,1.290412


## Pierce County

In [16]:
dist_by_mode('Work', 'Pierce')

source,model,survey
tmodetp,,
Bike,6.012893,4.134071
HOV2,13.927187,12.279696
HOV3+,14.436434,13.142913
Park,22.329932,33.238352
SOV,14.585771,12.322560
TNC,7.800935,28.521199
Transit,25.101874,24.670668
Walk,0.876831,1.117957


## Other Counties

In [17]:
dist_by_mode('Work', 'Pierce',True)

source,model,survey
tmodetp,,
Bike,5.724873,4.333523
HOV2,11.378654,11.087784
HOV3+,11.781283,15.496797
Park,14.604798,21.624501
SOV,11.924909,11.624631
TNC,6.000736,4.212889
Transit,11.572248,9.779546
Walk,0.747782,1.318596


### Average School Tour Distance by Mode

In [18]:
dist_by_mode('School')

source,model,survey
tmodetp,,
Bike,2.621351,1.295566
HOV2,3.570201,5.358035
HOV3+,3.652178,3.623373
Park,NaN,28.455959
SOV,7.511449,6.082790
School Bus,3.199445,3.464268
TNC,2.331336,2.772132
Transit,5.831225,7.658118
Walk,1.106157,0.948390


## Pierce County

In [19]:
dist_by_mode('School', 'Pierce')

source,model,survey
tmodetp,,
Bike,2.944644,NaN
HOV2,3.633576,3.018525
HOV3+,3.730074,4.657560
SOV,7.506713,5.675806
School Bus,3.267980,2.941240
TNC,2.386960,NaN
Transit,4.893098,16.751389
Walk,1.182998,1.318697


## Other Counties

In [20]:
dist_by_mode('School', 'Pierce', True)

source,model,survey
tmodetp,,
Bike,2.542567,1.295566
HOV2,3.552631,5.730542
HOV3+,3.630716,3.335890
Park,NaN,28.455959
SOV,7.512849,6.240134
School Bus,3.179997,3.663475
TNC,2.315946,2.772132
Transit,5.960078,7.504425
Walk,1.089053,0.911724


### Average Personal Business Tour Distance by Mode

In [21]:
dist_by_mode('Personal Business')

source,model,survey
tmodetp,,
Bike,3.307565,3.049242
HOV2,7.819294,6.479523
HOV3+,7.792973,6.014153
Park,NaN,19.282414
SOV,6.619172,5.332477
TNC,1.974820,4.930529
Transit,6.674110,6.090867
Walk,1.175463,0.899835


## Pierce County

In [22]:
dist_by_mode('Personal Business', 'Pierce')

source,model,survey
tmodetp,,
Bike,3.475647,1.500000
HOV2,8.057923,9.819904
HOV3+,7.943263,10.777941
Park,NaN,35.500000
SOV,6.880716,4.497669
TNC,2.273234,5.500000
Transit,8.072840,2.612107
Walk,1.293242,0.671420


## Other Counties

In [23]:
dist_by_mode('Personal Business', 'Pierce', True)

source,model,survey
tmodetp,,
Bike,3.260713,3.158012
HOV2,7.746820,5.756659
HOV3+,7.747423,5.246885
Park,NaN,12.628330
SOV,6.545809,5.551353
TNC,1.915979,4.864113
Transit,6.646815,6.775434
Walk,1.147493,0.935576


### Average Shopping Tour Distance by Mode

In [24]:
dist_by_mode('Shop')

source,model,survey
tmodetp,,
Bike,2.469323,0.561337
HOV2,5.528028,4.915027
HOV3+,5.680500,6.499530
Park,NaN,0.500000
SOV,4.725161,3.756609
TNC,1.416242,2.642004
Transit,4.358763,2.258857
Walk,1.054123,0.630863


## Pierce County

In [25]:
dist_by_mode('Shop', 'Pierce')

source,model,survey
tmodetp,,
Bike,2.947328,0.500000
HOV2,6.030779,5.044395
HOV3+,6.067596,4.475216
SOV,5.237160,4.174874
TNC,1.694004,3.500000
Transit,4.804348,2.295897
Walk,1.238543,0.565080


## Other Counties

In [26]:
dist_by_mode('Shop', 'Pierce', True)

source,model,survey
tmodetp,,
Bike,2.352060,0.562264
HOV2,5.380530,4.863512
HOV3+,5.566825,6.837103
Park,NaN,0.500000
SOV,4.584146,3.622121
TNC,1.371903,0.890120
Transit,4.353497,2.248567
Walk,1.019714,0.634649


### Average Meal Tour Distance by Mode

In [27]:
dist_by_mode('Meal')

source,model,survey
tmodetp,,
Bike,1.424779,2.683948
HOV2,8.745123,4.585905
HOV3+,9.335788,6.294448
Park,NaN,30.114636
SOV,5.023600,4.501964
TNC,1.385632,2.337364
Transit,5.894128,8.467957
Walk,0.648890,0.704359


## Pierce County

In [28]:
dist_by_mode('Meal', 'Pierce')

source,model,survey
tmodetp,,
Bike,1.470954,NaN
HOV2,9.070391,4.832888
HOV3+,9.540246,10.025907
Park,NaN,35.500000
SOV,5.226168,3.513033
TNC,1.405222,7.500000
Transit,5.013419,NaN
Walk,0.703087,0.758710


## Other Counties

In [29]:
dist_by_mode('Meal', 'Pierce', True)

source,model,survey
tmodetp,,
Bike,1.417677,2.683948
HOV2,8.666515,4.530467
HOV3+,9.286249,4.818639
Park,NaN,14.847628
SOV,4.976444,4.875622
TNC,1.383100,2.036600
Transit,5.943830,8.467957
Walk,0.641606,0.697717


### Average Social Tour Distance by Mode

In [30]:
dist_by_mode('Social')

source,model,survey
tmodetp,,
Bike,2.517048,2.334264
HOV2,6.044554,7.382173
HOV3+,6.040852,4.983953
Park,NaN,54.147252
SOV,5.359717,5.652026
TNC,1.406970,3.019705
Transit,5.166877,11.227402
Walk,0.952762,0.882807


## Pierce County

In [31]:
dist_by_mode('Social', 'Pierce')

source,model,survey
tmodetp,,
Bike,2.832248,6.873490
HOV2,6.123265,7.900494
HOV3+,6.093016,3.631786
SOV,5.456372,6.290074
TNC,1.638040,NaN
Transit,3.952381,4.774112
Walk,0.998335,0.932017


## Other Counties

In [32]:
dist_by_mode('Social', 'Pierce', True)

source,model,survey
tmodetp,,
Bike,2.441257,1.714584
HOV2,6.023501,7.324686
HOV3+,6.026505,5.316174
Park,NaN,54.147252
SOV,5.335852,5.520247
TNC,1.368747,3.019705
Transit,5.187364,11.442942
Walk,0.942070,0.869580


### Average Escort Tour Distance by Mode

In [33]:
dist_by_mode('Escort')

source,model,survey
tmodetp,,
Bike,12.523849,0.647842
HOV2,6.508272,4.363578
HOV3+,7.117393,3.749272
SOV,22.022213,3.974768
TNC,1.509073,0.500000
Transit,2.869942,2.256245
Walk,0.966994,0.552118


## Pierce County

In [34]:
dist_by_mode('Escort', 'Pierce')

source,model,survey
tmodetp,,
Bike,12.394243,NaN
HOV2,6.624954,3.680789
HOV3+,7.073893,5.711270
SOV,24.263664,5.351971
TNC,1.465753,NaN
Transit,2.282609,0.500000
Walk,1.042328,1.000000


## Other Counties

In [35]:
dist_by_mode('Escort', 'Pierce', True)

source,model,survey
tmodetp,,
Bike,12.560197,0.647842
HOV2,6.476096,4.603764
HOV3+,7.129759,3.103413
SOV,21.294082,3.412688
TNC,1.516548,0.500000
Transit,2.960000,2.347211
Walk,0.950152,0.549677


Survey Sample Size Reference by Mode and Purpose

In [36]:
# Survey Sample size
df_tour_survey = pd.read_csv(r'inputs/base_year/survey/_tour.tsv', delim_whitespace=True)

In [37]:
df_tour_survey.pivot_table(index='tmodetp',columns='pdpurp', values='toexpfac', aggfunc='count')

pdpurp,1,2,3,4,5,6,7,10
tmodetp,,,,,,,,
1,889.0,189.0,158.0,375.0,631.0,811.0,1613.0,NaN
2,234.0,38.0,8.0,19.0,19.0,19.0,81.0,NaN
3,3215.0,114.0,43.0,890.0,820.0,352.0,937.0,1.0
4,945.0,284.0,637.0,560.0,587.0,373.0,743.0,NaN
5,455.0,442.0,477.0,336.0,287.0,246.0,659.0,1.0
6,2293.0,257.0,26.0,253.0,146.0,115.0,314.0,1.0
7,101.0,2.0,NaN,7.0,1.0,4.0,10.0,NaN
8,9.0,253.0,1.0,20.0,NaN,NaN,7.0,NaN
9,119.0,12.0,1.0,29.0,6.0,42.0,49.0,NaN


Survey Sample Size Reference by Mode and Purpose 
(for Pierce County)

In [38]:
df_household = pd.read_csv(r'inputs/base_year/survey/_household.tsv', delim_whitespace=True)
df_household = df_household[['hhno','hhtaz']]

pierce_tazs = [i for i in range(2669, 3466+1)]
king_tazs = [i for i in range(1, 2088+1)]
kitsap_tazs = [i for i in range(3467, 3700+1)]
snohomish_tazs = [i for i in range(2089, 2668+1)]
df_household.loc[df_household['hhtaz'].isin(pierce_tazs), 'county'] = 'Pierce'
df_household.loc[df_household['hhtaz'].isin(king_tazs), 'county'] = 'King'
df_household.loc[df_household['hhtaz'].isin(kitsap_tazs), 'county'] = 'Kitsap'
df_household.loc[df_household['hhtaz'].isin(snohomish_tazs), 'county'] = 'Snohomish'

In [39]:
df_tour_survey = pd.merge(df_tour_survey, df_household, on="hhno", how='left')
for county, df_county in df_tour_survey.groupby('county'):
    print(county + ' County\n')
    print(df_tour_survey.pivot_table(index=['tmodetp'],columns='pdpurp', values='toexpfac', aggfunc='count', fill_value=0))
    print('\n')

King County

pdpurp     1    2    3    4    5    6     7   10
tmodetp                                         
1         889  189  158  375  631  811  1613   0
2         234   38    8   19   19   19    81   0
3        3215  114   43  890  820  352   937   1
4         945  284  637  560  587  373   743   0
5         455  442  477  336  287  246   659   1
6        2293  257   26  253  146  115   314   1
7         101    2    0    7    1    4    10   0
8           9  253    1   20    0    0     7   0
9         119   12    1   29    6   42    49   0
10        128   27    8   32   17   11    47   0


Kitsap County

pdpurp     1    2    3    4    5    6     7   10
tmodetp                                         
1         889  189  158  375  631  811  1613   0
2         234   38    8   19   19   19    81   0
3        3215  114   43  890  820  352   937   1
4         945  284  637  560  587  373   743   0
5         455  442  477  336  287  246   659   1
6        2293  257   26  253  146  115 

King County

pdpurp     1    2    3    4    5    6     7   10
tmodetp                                         
1         889  189  158  375  631  811  1613   0
2         234   38    8   19   19   19    81   0
3        3215  114   43  890  820  352   937   1
4         945  284  637  560  587  373   743   0
5         455  442  477  336  287  246   659   1
6        2293  257   26  253  146  115   314   1
7         101    2    0    7    1    4    10   0
8           9  253    1   20    0    0     7   0
9         119   12    1   29    6   42    49   0
10        128   27    8   32   17   11    47   0


Kitsap County

pdpurp     1    2    3    4    5    6     7   10
tmodetp                                         
1         889  189  158  375  631  811  1613   0
2         234   38    8   19   19   19    81   0
3        3215  114   43  890  820  352   937   1
4         945  284  637  560  587  373   743   0
5         455  442  477  336  287  246   659   1
6        2293  257   26  253  146  115 

# Work Tour Distance by Worker Type

In [40]:
df = load_data('tour_distance')
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df_dist = pd.DataFrame()
for source in ['model','survey']:
    _df = df[(df['source'] == source) & 
             (df['pdpurp'] == 'Work') & 
             (df['pptyp'].isin(['Full-Time Worker','Part-Time Worker','University Student']))]
    df1 = _df.groupby('pptyp').sum()[['wt_autodist']].reset_index()
    df2 = _df.groupby('pptyp').sum()[['toexpfac']].reset_index()
    _df = df1.merge(df2, on='pptyp')
    _df['commute_distance'] = _df['wt_autodist']/_df['toexpfac']
    _df['source'] = source
    df_dist = df_dist.append(_df)
df = df_dist.pivot_table(index='pptyp', columns='source', aggfunc='first', values='commute_distance').reset_index()
df.rename(columns={'pptyp': 'Person Type'}, inplace=True)
df

source,Person Type,model,survey
0,Full-Time Worker,12.271586,12.146355
1,Part-Time Worker,8.941532,7.607614
2,University Student,10.081480,9.322525


## Pierce County

In [41]:
df = load_data('tour_distance')
df = df.loc[df.person_county=='Pierce']
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df_dist = pd.DataFrame()
for source in ['model','survey']:
    _df = df[(df['source'] == source) & 
             (df['pdpurp'] == 'Work') & 
             (df['pptyp'].isin(['Full-Time Worker','Part-Time Worker','University Student']))]
    df1 = _df.groupby('pptyp').sum()[['wt_autodist']].reset_index()
    df2 = _df.groupby('pptyp').sum()[['toexpfac']].reset_index()
    _df = df1.merge(df2, on='pptyp')
    _df['commute_distance'] = _df['wt_autodist']/_df['toexpfac']
    _df['source'] = source
    df_dist = df_dist.append(_df)
df = df_dist.pivot_table(index='pptyp', columns='source', aggfunc='first', values='commute_distance').reset_index()
df.rename(columns={'pptyp': 'Person Type'}, inplace=True)
df

source,Person Type,model,survey
0,Full-Time Worker,15.519242,14.398612
1,Part-Time Worker,9.809447,5.253085
2,University Student,11.615299,15.193166


## Other Counties

In [42]:
df = load_data('tour_distance')
df = df.loc[df.person_county!='Pierce']
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df_dist = pd.DataFrame()
for source in ['model','survey']:
    _df = df[(df['source'] == source) & 
             (df['pdpurp'] == 'Work') & 
             (df['pptyp'].isin(['Full-Time Worker','Part-Time Worker','University Student']))]
    df1 = _df.groupby('pptyp').sum()[['wt_autodist']].reset_index()
    df2 = _df.groupby('pptyp').sum()[['toexpfac']].reset_index()
    _df = df1.merge(df2, on='pptyp')
    _df['commute_distance'] = _df['wt_autodist']/_df['toexpfac']
    _df['source'] = source
    df_dist = df_dist.append(_df)
df = df_dist.pivot_table(index='pptyp', columns='source', aggfunc='first', values='commute_distance').reset_index()
df.rename(columns={'pptyp': 'Person Type'}, inplace=True)
df

source,Person Type,model,survey
0,Full-Time Worker,11.593370,11.658014
1,Part-Time Worker,8.761465,8.829920
2,University Student,9.716132,8.405636
